In [ ]:
%autosave 600

In [1]:
import numpy as np
import pandas as pd

from scipy.stats import rankdata
from sklearn.metrics import roc_auc_score

import os 
from glob import glob

train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

classes = train.columns[-6:]
print(classes)


Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')


In [2]:
from sklearn.preprocessing import normalize

In [3]:

files = glob(os.path.join(os.pardir, 'ensembles', 'pred*.csv'))
files = sorted(files)
files

['../ensembles/prediction_Capsule.csv',
 '../ensembles/prediction_DPCNN.csv',
 '../ensembles/prediction_LSTM.csv',
 '../ensembles/prediction_lgbm.csv',
 '../ensembles/preds_bilstmft.csv',
 '../ensembles/preds_log_reg_sgd.csv',
 '../ensembles/preds_logreg_liblinear.csv',
 '../ensembles/preds_logreg_sanket.csv',
 '../ensembles/preds_lsvc_ftrl.csv',
 '../ensembles/preds_lsvc_liblinear.csv',
 '../ensembles/preds_nb_svm.csv']

In [4]:
from sklearn.preprocessing import minmax_scale
preds_list = []
for i, f in enumerate(files):
    vals = pd.read_csv(f, usecols = classes).values
    vals = minmax_scale(vals)
    preds_list.append(vals)

In [ ]:

def get_preds(coeffs):
    preds = np.zeros_like(preds_list[0])
    for c,p in zip(coeffs, preds_list):
        for i in range(len(classes)):
            preds[:, i] = np.add(preds[:, i], c*(p[:, i]))
    preds /= len(preds_list)
    labels = train[classes].values
    j = 1* roc_auc_score(labels, preds)
    # print(coeffs, j, '\n')
    return j
                   
    
carray = np.random.randint(0, 9, (1500, len(files)))
ncarray = normalize(carray, norm='l1') 
h = []

for i in range(carray.shape[0]):
    q = get_preds(carray[i])
    h.append(q)

max = np.array(h).argmax()
print(h[max], '\n', ncarray[max])

In [16]:
max = np.array(h).argmax()
print(h[max], '\n', ncarray[max])

0.9912427698542107 
 [0.2173913  0.13043478 0.08695652 0.         0.08695652 0.08695652
 0.08695652 0.08695652 0.         0.         0.2173913 ]


In [29]:
files2 = glob(os.path.join(os.pardir, 'ensembles', 'test*.csv'))
files2=sorted(files2)
files2


['../ensembles/test_average_Capsule.csv',
 '../ensembles/test_average_DPCNN.csv',
 '../ensembles/test_average_LSTM.csv',
 '../ensembles/test_average_lbgm.csv',
 '../ensembles/test_bilstmft.csv',
 '../ensembles/test_log_reg_sgd.csv',
 '../ensembles/test_logreg_liblinear.csv',
 '../ensembles/test_logreg_sanket.csv',
 '../ensembles/test_lsvc_ftrl.csv',
 '../ensembles/test_lsvc_liblinear.csv',
 '../ensembles/test_nb_svm.csv']

In [30]:
from sklearn.preprocessing import minmax_scale
sub_list = []
for i, f in enumerate(files2):
    vals = pd.read_csv(f, usecols = classes).values
#     vals = minmax_scale(vals)
    sub_list.append(vals)

In [31]:
# coeffs = ncarray[max]
coeffs =  np.array([0.2173913, 0.13043478, 0.08695652, 0., 0.08695652, 0.08695652,
 0.08695652, 0.08695652, 0.,  0., 0.2173913])

sub = np.zeros_like(sub_list[0])
for c,p in zip(coeffs, sub_list):
    for i in range(len(classes)):
        sub[:, i] = np.add(sub[:, i], c*(p[:, i]))
sub /= len(sub_list)


In [32]:
sub.shape

(153164, 6)

In [33]:
submission = pd.read_csv("../input/sample_submission.csv")
# submission[classes] = minmax_scale(preds)
submission[classes] = sub
submission.to_csv("../subs/ens_10models_minmax400.csv", index = False)